In [1]:
%pip install -U transformers datasets==2.21.0 scipy scikit-learn

Note: you may need to restart the kernel to use updated packages.


# Load Data

In [2]:
from datasets import load_dataset

task = "ynat"  # Yahoo News Article Topic Classification
datasets = load_dataset("klue", task)  # Korean Language Understanding Evaluation

In [3]:
datasets

DatasetDict({
    train: Dataset({
        features: ['guid', 'title', 'label', 'url', 'date'],
        num_rows: 45678
    })
    validation: Dataset({
        features: ['guid', 'title', 'label', 'url', 'date'],
        num_rows: 9107
    })
})

In [4]:
datasets["train"][0]

{'guid': 'ynat-v1_train_00000',
 'title': '유튜브 내달 2일까지 크리에이터 지원 공간 운영',
 'label': 3,
 'url': 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=227&oid=001&aid=0008508947',
 'date': '2016.06.30. 오전 10:36'}

In [5]:
import pandas as pd
from IPython.display import display, HTML
from datasets import ClassLabel
import random
import numpy as np


def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."

    picks = []

    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)

        # 이미 등록된 예제가 뽑힌 경우, 다시 추출
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)

        picks.append(pick)

    # 임의로 추출된 인덱스들로 구성된 데이터 프레임 선언
    df = pd.DataFrame(dataset[picks])

    for column, typ in dataset.features.items():
        # 라벨 클래스를 스트링으로 변환
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])

    display(HTML(df.to_html()))

In [6]:
show_random_elements(datasets["train"])

,guid,title,label,url,date
0,ynat-v1_train_16018,中 지난달 차이신 제조업 PMI 49.7…1년반만에 경기위축 진입종합,세계,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=104&sid2=231&oid=001&aid=0010557853,2019.01.02. 오전 11:54
1,ynat-v1_train_14558,신간 마블이 설계한 사소하고 위대한 과학,생활문화,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=103&sid2=243&oid=001&aid=0011195712,2019.11.07. 오전 10:39
2,ynat-v1_train_41528,1보 코스피 엿새째 하락 1910선 내줘…코스닥은 2%대 상승,경제,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=258&oid=001&aid=0011007136,2019.08.07. 오후 3:35
3,ynat-v1_train_44837,그래픽 지난해 51채 이상 집부자 1988명,경제,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=263&oid=001&aid=0010542037,2018.12.23. 오후 4:11
4,ynat-v1_train_18671,신간 거짓말 읽는 법·사람의 자리 과학의 마음에 닿다,생활문화,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=103&sid2=243&oid=001&aid=0010773097,2019.04.19. 오전 7:00
5,ynat-v1_train_00507,셰크먼 호기심 분야 꾸준한 연구가 노벨상 비결,IT과학,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=228&oid=001&aid=0008729806,2016.10.05. 오후 2:58
6,ynat-v1_train_09046,IoT로 쓰레기통 관리해요,IT과학,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=227&oid=001&aid=0009270846,2017.05.17. 오전 10:40
7,ynat-v1_train_42212,금성테크 지에스알파트너스에 의해 파산신청…거래 정지,경제,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=261&oid=001&aid=0008501668,2016.06.27. 오후 5:56
8,ynat-v1_train_42518,게시판 신한금융 기후변화 대응 최우수기업으로 명예의 전당 입성,경제,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=259&oid=001&aid=0010783342,2019.04.24. 오전 11:24
9,ynat-v1_train_35631,카톡 이모티콘 카카오 TV·다음 포털에서도 쓴다,IT과학,https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=227&oid=001&aid=0009198040,2017.04.18. 오후 2:00


**KLUE TC**

- 0 (IT과학)
- 1 (경제)
- 2 (사회)
- 3 (생활문화)
- 4 (세계)
- 5 (스포츠)
- 6 (정치)

# metric 설정

In [7]:
from datasets import load_metric

metric = load_metric("f1", trust_remote_code=True)

/var/folders/h0/f86w2r851zn7yj1w80yw932h0000gn/T/ipykernel_21753/3905583055.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("f1", trust_remote_code=True)


In [8]:
fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))

fake_preds, fake_labels

(array([0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0,
        1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1]),
 array([0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
        1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0]))

In [9]:
metric.compute(
    predictions=fake_preds,  # 예측 값
    references=fake_labels,  # target
)

{'f1': 0.6666666666666666}

# Tokenizer 설정

In [10]:
# 한국어 NLU를 위한 대표적인 모델
model_name = "klue/roberta-base"

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [12]:
query = "프리미어 리그에서 활약할 손흥민 선수의 마지막 한국 인터뷰"

In [13]:
tokenizer(query)

{'input_ids': [0, 12416, 4469, 27135, 5943, 2085, 11251, 3825, 2079, 4178, 3629, 5111, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
tokenizer.cls_token_id

0

In [15]:
tokenizer

BertTokenizerFast(name_or_path='klue/roberta-base', vocab_size=32000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [16]:
# 데이터 전처리 함수 정의
def preprocess_function(data):
    return tokenizer(
        data["title"],  # 데이터
        truncation=True,  # 문장이 모델이 받아들일 수 있는 최대 길이 이상으로 들어올 경우 최대 길이를 기준으로 시퀀스(문장) 자르기
        return_token_type_ids=False,  # token_type_ids가 필요없는 RoBERTa 모델은 False로 설정
    )

In [17]:
preprocess_function(datasets["train"][:3])

{'input_ids': [[0, 10637, 8474, 22, 2210, 2299, 2118, 28940, 3691, 4101, 3792, 2], [0, 24905, 1042, 4795, 19982, 2129, 121, 6904, 16311, 3, 14392, 2], [0, 4172, 3797, 3728, 2107, 2134, 3777, 904, 6022, 2332, 2113, 2259, 4523, 1380, 2259, 2062, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [18]:
# 모든 데이터에 대해 전처리 적용
encoded_datasets = datasets.map(preprocess_function, batched=True)

# Model 설정

In [19]:
from transformers import AutoModelForSequenceClassification

num_labels = 7
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=num_labels
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
# 모델이 예측 했을 때 지표 계산
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    return metric.compute(predictions=predictions, references=labels, average="macro")

## Trainer 정의
이제 앞서 정의한 정보들을 바탕으로 `transformers`에서 제공하는 *Trainer* 객체를 활용하기 위한 인자 관리 클래스를 초기화합니다.

`metric_name`은 앞서 얻어진 메트릭 함수를 활용했을 때, 아래와 같이 `dict` 형식으로 결과 값이 반환되는데 여기서 우리가 사용할 *key* 를 정의해준다고 생각하시면 됩니다.

```python
>>> metric.compute(predictions=fake_preds, references=fake_labels)
{'f1': 0.85}
```

각 인자에 대한 자세한 설명은 [문서](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments)에서 참조해주시면 됩니다.

In [21]:
%pip install "transformers[torch]" "accelerate>=0.26.0"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [22]:
from transformers import (
    TrainingArguments,
)  # Trainer 클래스에서 사용할 훈련용 하이퍼 파라미터 정의

metric_name = "f1"  # metric 객체와 동일한 지표를 사용
batch_size = 32

args = TrainingArguments(
    "../../data/saved_models/test-tc",  # 모델 결과물을 저장할 디렉토리
    evaluation_strategy="epoch",  # 평가 전략. 언제 마다 평가 할지를 결정
    save_strategy="epoch",  # 모델 저장은 언제마다 할건지.
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

/opt/anaconda3/envs/llm-env/lib/python3.11/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [23]:
from transformers import Trainer  # TrainingArguments를 입력 받아 훈련을 수행

trainer = Trainer(
    model,  # 훈련 시킬 모델
    args,  # TrainingArguments
    train_dataset=encoded_datasets["train"],
    eval_dataset=encoded_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

  0%|          | 0/7140 [00:00<?, ?it/s]

{'loss': 0.5628, 'grad_norm': 9.216388702392578, 'learning_rate': 1.8599439775910366e-05, 'epoch': 0.35}
{'loss': 0.39, 'grad_norm': 6.487459182739258, 'learning_rate': 1.719887955182073e-05, 'epoch': 0.7}


  0%|          | 0/285 [00:00<?, ?it/s]

{'eval_loss': 0.4037039279937744, 'eval_f1': 0.8606807733675929, 'eval_runtime': 33.5434, 'eval_samples_per_second': 271.499, 'eval_steps_per_second': 8.496, 'epoch': 1.0}
{'loss': 0.3572, 'grad_norm': 5.403110980987549, 'learning_rate': 1.5798319327731094e-05, 'epoch': 1.05}
{'loss': 0.2895, 'grad_norm': 4.995489120483398, 'learning_rate': 1.4397759103641458e-05, 'epoch': 1.4}
{'loss': 0.2948, 'grad_norm': 5.401579856872559, 'learning_rate': 1.2997198879551822e-05, 'epoch': 1.75}


  0%|          | 0/285 [00:00<?, ?it/s]

...

  0%|          | 0/285 [00:00<?, ?it/s]

{'eval_loss': 0.45816242694854736, 'eval_f1': 0.864923492036132, 'eval_runtime': 34.8216, 'eval_samples_per_second': 261.533, 'eval_steps_per_second': 8.185, 'epoch': 4.0}
{'loss': 0.1436, 'grad_norm': 7.979711055755615, 'learning_rate': 3.1932773109243696e-06, 'epoch': 4.2}
{'loss': 0.1297, 'grad_norm': 3.0681309700012207, 'learning_rate': 1.792717086834734e-06, 'epoch': 4.55}
{'loss': 0.1155, 'grad_norm': 12.434139251708984, 'learning_rate': 3.921568627450981e-07, 'epoch': 4.9}


  0%|          | 0/285 [00:00<?, ?it/s]

{'eval_loss': 0.49951910972595215, 'eval_f1': 0.862966249587739, 'eval_runtime': 37.6398, 'eval_samples_per_second': 241.951, 'eval_steps_per_second': 7.572, 'epoch': 5.0}
{'train_runtime': 5672.0255, 'train_samples_per_second': 40.266, 'train_steps_per_second': 1.259, 'train_loss': 0.24900020524567248, 'epoch': 5.0}


TrainOutput(global_step=7140, training_loss=0.24900020524567248, metrics={'train_runtime': 5672.0255, 'train_samples_per_second': 40.266, 'train_steps_per_second': 1.259, 'total_flos': 2555331086520900.0, 'train_loss': 0.24900020524567248, 'epoch': 5.0})

# Evaluate

In [24]:
# 가장 좋은 Metric 을 보인 Model 의 checkpoint
trainer.evaluate()

  0%|          | 0/285 [00:00<?, ?it/s]

{'eval_loss': 0.3689662516117096,
 'eval_f1': 0.8666661157253167,
 'eval_runtime': 39.7928,
 'eval_samples_per_second': 228.86,
 'eval_steps_per_second': 7.162,
 'epoch': 5.0}

# Pipeline 생성

In [26]:
from transformers import pipeline

classifier = pipeline(
    "text-classification",
    model="../../data/saved_models/test-tc/checkpoint-2856",
    return_all_scores=True,
)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/opt/anaconda3/envs/llm-env/lib/python3.11/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [27]:
print(query)

프리미어 리그에서 활약할 손흥민 선수의 마지막 한국 인터뷰


In [28]:
classifier("기안 84가 갠지스 강물을 마셨습니다.")

[[{'label': 'LABEL_0', 'score': 0.004838953725993633},
  {'label': 'LABEL_1', 'score': 0.0013362442841753364},
  {'label': 'LABEL_2', 'score': 0.02681775577366352},
  {'label': 'LABEL_3', 'score': 0.7445481419563293},
  {'label': 'LABEL_4', 'score': 0.21785353124141693},
  {'label': 'LABEL_5', 'score': 0.0016611474566161633},
  {'label': 'LABEL_6', 'score': 0.002944271545857191}]]